In [1]:
import copy
import sys
import numpy as np
import open3d as o3d
sys.path.append(".")
import pyransac3d as pyrsc
import flattten_pcd as fp
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-09-17 21:22:28,428 - topics - topicmanager initialized


In [2]:
load_pcd = o3d.io.read_point_cloud("point_pipe.pcd")
print("length pcd ", len(np.array(load_pcd.points)))
o3d.visualization.draw_geometries([load_pcd])

length pcd  9347


In [3]:
print("Downsample the point cloud with a voxel of 0.02")
pcd2 = load_pcd.voxel_down_sample(voxel_size=0.02)
print("length downpcd ", len(np.array(pcd2.points)))
o3d.visualization.draw_geometries([pcd2])

Downsample the point cloud with a voxel of 0.02
length downpcd  2627


In [4]:
plane_model, inliers = pcd2.segment_plane(distance_threshold=0.05,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd2.select_by_index(inliers)
print(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd2.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0, 1.0, 0])
print(len(inliers), len(np.array(pcd2.points)))

Plane equation: -0.00x + 0.00y + 1.00z + 0.01 = 0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 183, 184, 190, 191, 192, 193, 194, 195, 196, 198, 199, 200, 201, 203, 205, 206, 209, 210, 211, 212, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 225, 226, 228, 231, 232, 235, 241, 242, 247, 253, 255, 256, 257, 258, 260, 261, 262, 263, 264, 266, 267, 268, 269, 270, 271, 272, 273, 274, 277, 278, 279, 280, 281, 283, 284, 285, 286, 288, 290, 291, 292, 299, 30

In [5]:
print("inlider")
o3d.visualization.draw_geometries([inlier_cloud])
print("outlier_cloud")
o3d.visualization.draw_geometries([outlier_cloud])

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(outlier_cloud.cluster_dbscan(eps=0.1, min_points=100, print_progress=True))

inlider
outlier_cloud
[Open3D DEBUG] Precompute neighbors.
Precompute neighbors.[========================================] 100%
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2


In [6]:
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
print("length label:  ",len(labels))
print("length outlier_cloud:  ",len(np.array(outlier_cloud.points)))
o3d.visualization.draw_geometries([outlier_cloud])

point cloud has 2 clusters
length label:   943
length outlier_cloud:   943


In [7]:
cluster = o3d.geometry.PointCloud()
for c in range(max_label+1):
    cluster = [i for i, n in enumerate(labels.tolist()) if n == c]
    print(cluster) 
    cluster = outlier_cloud.select_by_index(cluster)
    print("length cluster:  ",len(np.array(cluster.points)))
    o3d.visualization.draw_geometries([cluster])

[2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 60, 74, 75, 76, 77, 78, 79, 80, 81, 82, 86, 87, 90, 91, 92, 93, 94, 95, 96, 99, 100, 101, 104, 105, 107, 108, 109, 110, 111, 112, 113, 115, 117, 118, 119, 121, 123, 124, 126, 127, 129, 130, 131, 132, 133, 134, 135, 136, 138, 139, 141, 143, 144, 147, 148, 149, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 162, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 177, 178, 183, 184, 185, 186, 189, 190, 191, 193, 194, 196, 197, 198, 199, 200, 201, 202, 203, 206, 207, 210, 212, 214, 215, 216, 217, 218, 220, 221, 225, 226, 227, 228, 229, 230, 231, 232, 238, 240, 241, 242, 243, 244, 246, 247, 249, 251, 252, 257, 259, 260, 261, 262, 264, 265, 268, 269, 270, 271, 273, 274, 276, 278, 279, 280, 281, 282, 284, 287, 288, 290, 293, 294, 295, 297, 300, 301, 304, 305, 306, 307, 308, 311, 312, 313, 314, 31

In [8]:
cluster = [i for i, n in enumerate(labels.tolist()) if n == 1]
print(cluster) 
cluster = outlier_cloud.select_by_index(cluster)
print("length cluster:  ",len(np.array(cluster.points)))
o3d.visualization.draw_geometries([cluster])

[55, 59, 63, 65, 66, 67, 68, 69, 71, 72, 83, 85, 98, 137, 140, 142, 145, 146, 150, 164, 258, 267, 277, 298, 309, 334, 340, 349, 368, 391, 402, 427, 432, 438, 443, 446, 451, 471, 472, 473, 474, 475, 477, 478, 479, 480, 481, 482, 483, 485, 487, 491, 492, 519, 523, 525, 527, 530, 532, 533, 534, 536, 537, 539, 552, 578, 579, 580, 584, 585, 588, 590, 591, 592, 594, 596, 597, 625, 629, 630, 634, 636, 639, 656, 657, 659, 661, 663, 665, 666, 667, 669, 671, 673, 674, 701, 702, 704, 725, 727, 729, 730, 731, 732, 733, 734, 744, 781, 790, 809, 826, 838, 849, 880, 882, 884, 923]
length cluster:   117


In [10]:
import cylinder2 as cy2

ModuleNotFoundError: No module named 'aux_functions'

In [11]:
cil = cy2.Cylinder()
points = np.asarray(cluster.points)
center, normal, radius, inliers = cil.fit(points, thresh=0.01, maxIteration=10000)

#center, normal, radius, inliers = fp.get_cylinder(cluster, thresh=0.01, maxIteration=10000)
mesh_cylinder = fp.get_clylinder_mesh(center, normal, radius)
print("center: " + str(center))
print("radius: " + str(radius))
print("vecC: " + str(normal))
#print("inliers: ", inliers)

inlier_cloud = cluster.select_by_index(inliers)

NameError: name 'cy2' is not defined

In [16]:
o3d.visualization.draw_geometries([mesh_cylinder,inlier_cloud])

NameError: name 'mesh_cylinder' is not defined